<a href="https://colab.research.google.com/github/ronitavalani/467Project/blob/main/genre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Step 1: Imports
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Step 2: Load and preprocess dataset
url = 'https://raw.githubusercontent.com/ronitavalani/467Project/main/songs_normalize.csv'
df = pd.read_csv(url)

# Keep first genre only
df['genre'] = df['genre'].astype(str).apply(lambda x: x.split(',')[0].strip())

# Drop non-numeric columns (except genre)
non_numeric_cols = df.select_dtypes(include=['object']).columns.tolist()
non_numeric_cols.remove('genre')
df = df.drop(columns=non_numeric_cols)

# Drop NA rows
df = df.dropna()

# Feature matrix and labels
X = df.drop(columns=['genre'])
y = df['genre']

# Drop genre classes with <2 examples
le = LabelEncoder()
y_encoded = le.fit_transform(y)
value_counts = pd.Series(y_encoded).value_counts()
valid_classes = value_counts[value_counts > 1].index
valid_mask = pd.Series(y_encoded).isin(valid_classes)
X = X[valid_mask]
y = y[valid_mask].reset_index(drop=True)

# Re-encode after filtering
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Step 3: Custom Dataset
class SongDataset(Dataset):
    def __init__(self, features, labels):
        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = SongDataset(X_train, y_train)
test_dataset = SongDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Step 4: Neural Network with One Hidden Layer
class GenreNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GenreNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Model, loss, optimizer
input_dim = X_train.shape[1]
hidden_dim = 64
output_dim = len(np.unique(y_encoded))

model = GenreNet(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Step 5: Training Loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Step 6: Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"\nTest Accuracy: {100 * correct / total:.2f}%")

Epoch 1/20, Loss: 1.9561
Epoch 2/20, Loss: 1.3303
Epoch 3/20, Loss: 1.1162
Epoch 4/20, Loss: 1.0425
Epoch 5/20, Loss: 1.0102
Epoch 6/20, Loss: 0.9920
Epoch 7/20, Loss: 0.9769
Epoch 8/20, Loss: 0.9634
Epoch 9/20, Loss: 0.9540
Epoch 10/20, Loss: 0.9441
Epoch 11/20, Loss: 0.9352
Epoch 12/20, Loss: 0.9259
Epoch 13/20, Loss: 0.9178
Epoch 14/20, Loss: 0.9110
Epoch 15/20, Loss: 0.9029
Epoch 16/20, Loss: 0.8951
Epoch 17/20, Loss: 0.8878
Epoch 18/20, Loss: 0.8818
Epoch 19/20, Loss: 0.8752
Epoch 20/20, Loss: 0.8686

Test Accuracy: 66.00%


In [5]:
# Step 1: Import libraries and load data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the CSV file
url = 'https://raw.githubusercontent.com/ronitavalani/467Project/main/songs_normalize.csv'
df = pd.read_csv(url)

# Step 2: Preprocess the genre column
df['genre'] = df['genre'].astype(str).apply(lambda x: x.split(',')[0].strip())

# Drop rows with missing values just in case
df = df.dropna()

# Step 3: Check genre distribution
print("Genre distribution:\n", df['genre'].value_counts())

# Drop non-numeric columns except 'genre'
non_numeric_cols = df.select_dtypes(include=['object']).columns.tolist()
non_numeric_cols.remove('genre')  # keep genre as label
X = df.drop(columns=non_numeric_cols + ['genre'])  # drop all other string columns
y = df['genre']

# Encode genre labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Drop classes with fewer than 2 samples (needed for stratify split)
genre_counts = pd.Series(y_encoded).value_counts()
valid_classes = genre_counts[genre_counts > 1].index
valid_mask = pd.Series(y_encoded).isin(valid_classes)

X_scaled = X_scaled[valid_mask]
y_filtered = y[valid_mask].reset_index(drop=True)  # Use original genre names, then re-encode
le = LabelEncoder()
y_encoded = le.fit_transform(y_filtered)

# Recalculate class weights after filtering
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_encoded),
    y=y_encoded
)
class_weights = dict(enumerate(weights))

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# Build basic neural network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    class_weight=class_weights
)

# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {accuracy:.2f}")


Genre distribution:
 genre
pop                  936
hip hop              776
rock                 162
Dance/Electronic      41
set()                 22
latin                 15
R&B                   13
country               11
World/Traditional     10
metal                  9
Folk/Acoustic          4
easy listening         1
Name: count, dtype: int64
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0893 - loss: 2.3990 - val_accuracy: 0.1000 - val_loss: 2.3686
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1012 - loss: 2.5405 - val_accuracy: 0.0531 - val_loss: 2.3690
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1007 - loss: 2.4176 - val_accuracy: 0.0469 - val_loss: 2.3548
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0942 - loss: 2.3547 - val_accuracy: 0.0719 - val_loss: 2.3449
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1038 - loss: 2.1877 - val_accuracy: 0.1031 - val_loss: 2.3008
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1247 - loss: 2.4140 - val_accuracy: 0.1250 - val_loss: 2.2966
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1544 - loss: 2.3603 - val_accuracy: 0.1969 - val_loss: 2.2496
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2069 - loss: 2.2533 - val_accuracy: 0.2125 - val_loss: 2.2093
Epo